In [ ]:
!pip install crewai google-generativeai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.4/80.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of instructor to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 4.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-grpc to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 665.2/665.2 kB 15.7

In [ ]:
# Get API keys from environment variables
from google.colab import userdata
import google.generativeai as genai
import os

GEMINI_API_KEY = userdata.get("GOOGLE_API_KEY")
if not GEMINI_API_KEY:
    print("Warning: GEMINI_API_KEY not found. Gemini model will not run.")

genai.configure(api_key=GEMINI_API_KEY)

In [ ]:
"""
Project 1: Multi-Agent Email Writer

Crew Setup

Agent 1 → Understand user request

Agent 2 → Rewrite email professionally
"""

'\nProject 1: Multi-Agent Email Writer\n\nCrew Setup\n\nAgent 1 → Understand user request\n\nAgent 2 → Rewrite email professionally\n'

In [ ]:
from crewai import Agent, Task, Crew, LLM, Process
from google.generativeai import GenerativeModel

llm = LLM(
    model="gemini-2.5-flash",
    temperature=0.1,
    api_key=GEMINI_API_KEY
)

"""
llm1 = LLM(
    model="gemini-2.0-flash",
    temperature=0.1,
    api_key=GEMINI_API_KEY
)
"""


'\nllm1 = LLM(\n    model="gemini-2.0-flash",\n    temperature=0.1,\n    api_key=GEMINI_API_KEY\n)\n'

In [ ]:
writer = Agent(
    role="Professional Email Writer",
    goal="Write polished, formal emails based on user needs.",
    backstory=(
        "You are an experienced corporate communication specialist who writes "
        "clear and professional emails."
    ),
    llm=llm,
   # verbose=True,
    allow_delegation=False,
)

reviewer = Agent(
    role="Email Reviewer",
    goal="Review emails for grammar, tone, and clarity and improve them.",
    backstory=(
        "You are a senior editor who refines wording and ensures professional tone."
    ),
    llm=llm,
   # verbose=True,
    allow_delegation=False,
)

In [ ]:
# Task (assigned to writer;)
email_task = Task(
    description=(
        "Write a formal email to your manager requesting 2 days of sick leave due "
        "to fever and doctor's advice to rest. Keep it concise and polite."
    ),
    expected_output="A short, formal, well-structured email.",
    agent=writer,
)

In [ ]:
crew = Crew(
    agents=[writer, reviewer],
    tasks=[email_task],
    process=Process.sequential,  # default is sequential; explicit for clarity
    verbose=True,
)

result = crew.kickoff()
print("\n=== FINAL EMAIL ===\n")
print(result)


=== FINAL EMAIL ===

Subject: Sick Leave Request - [Your Name]

Dear [Manager's Name],

I am writing to request two days of sick leave on [Start Date] and [End Date]. I have developed a fever and have been advised by a doctor to rest completely.

I will be unavailable during this period and will ensure to catch up on any missed work upon my return.

Thank you for your understanding.

Sincerely,
[Your Name]


╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): 



In [ ]:
# Task 1 → Writer creates initial email
task_write = Task(
    description=(
        "Write a short formal email requesting 2 days sick leave due to fever and "
        "doctor's recommendation to rest."
    ),
    expected_output="A complete polished email in formal tone.",
    agent=writer
)

# Task 2 → Reviewer improves email from Task 1
task_review = Task(
    description=(
        "Improve the email written previously. Correct grammar, improve tone and "
        "make wording more professional but concise."
    ),
    expected_output="Final refined email ready to send.",
    agent=reviewer
)

crew = Crew(
    agents=[writer, reviewer],
    tasks=[task_write, task_review],
    process=Process.sequential,
    verbose=True
)

result = crew.kickoff()

print("\n=== FINAL EMAIL ===\n")
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0aa7de78-de20-4d4d-be98-b988961ba6a7                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Professional Email Writer                                                                               │
│                                                                                                                 │
│  Task: Write a short formal email requesting 2 days sick leave due to fever and doctor's recommendation to      │
│  rest.                                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Professional Email Writer                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Subject: Sick Leave Request - [Your Name]                                                                      │
│                                                                                                                 │
│  Dear [Manager's Name],                                                                                         │
│                                                                                                                 │
│  I am writing to formally request two days of sick leave on [Start Date] and [End Date]. I have developed a     │
│  fever and have been advised by my doctor to rest for the next two days to ensure a full recovery.              │
│                                                                                                                 │
│  I will monitor my email periodically for urgent matters, health permitting, and will ensure any critical       │
│  tasks are addressed upon my return.                                                                            │
│                                                                                                                 │
│  Thank you for your understanding.                                                                              │
│                                                                                                                 │
│  Sincerely,                                                                                                     │
│                                                                                                                 │
│  [Your Name]                                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: f8decd5b-3137-4c24-b172-172f0b553f58                                                                     │
│  Agent: Professional Email Writer                                                                               │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Reviewer                                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Subject: Sick Leave Request - [Your Name]                                                                      │
│                                                                                                                 │
│  Dear [Manager's Name],                                                                                         │
│                                                                                                                 │
│  I am writing to formally request two days of sick leave on [Start Date] and [End Date]. I am experiencing a    │
│  fever and have been medically advised to rest for the next two days for a full recovery.                       │
│                                                                                                                 │
│  I will periodically check emails for urgent matters, as my health allows, and will prioritize critical tasks   │
│  upon my return.                                                                                                │
│                                                                                                                 │
│  Thank you for your understanding.                                                                              │
│                                                                                                                 │
│  Sincerely,                                                                                                     │
│                                                                                                                 │
│  [Your Name]                                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 673788e9-971a-4f20-8c6c-1903596e7670                                                                     │
│  Agent: Email Reviewer                                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


=== FINAL EMAIL ===

Subject: Sick Leave Request - [Your Name]

Dear [Manager's Name],

I am writing to formally request two days of sick leave on [Start Date] and [End Date]. I am experiencing a fever and have been medically advised to rest for the next two days for a full recovery.

I will periodically check emails for urgent matters, as my health allows, and will prioritize critical tasks upon my return.

Thank you for your understanding.

Sincerely,

[Your Name]


╭────────────────────────────── Execution Traces ──────────────────────────────╮
│                                                                              │
│  🔍 Detailed execution traces are available!                                 │
│                                                                              │
│  View insights including:                                                    │
│    • Agent decision-making process                                           │
│    • Task execution flow and timing    

## 3-agent Email & Document Review Crew

In [ ]:
from crewai import Agent

draft_improver = Agent(
    role="Draft Improver",
    goal=(
        "Take rough, unstructured text and turn it into a clear, complete draft "
        "while preserving the original meaning and intent."
    ),
    backstory=(
        "You are a helpful writing assistant who focuses on clarity, completeness, "
        "and eliminating confusion from the initial draft."
    ),
    llm=llm,
    verbose=True,
    allow_delegation=False,
)

editor = Agent(
    role="Editor",
    goal=(
        "Improve structure, tone, and readability of a draft while keeping the "
        "author's voice and intent intact."
    ),
    backstory=(
        "You are an experienced editor who works on emails, articles, and reports, "
        "making them sound professional and engaging."
    ),
    llm=llm,
    verbose=True,
    allow_delegation=False,
)

proofreader = Agent(
    role="Proofreader",
    goal=(
        "Catch grammar, spelling, punctuation, and minor style issues in text "
        "that is already mostly polished."
    ),
    backstory=(
        "You are a meticulous proofreader who focuses on the final polish before "
        "the text is sent or published."
    ),
    llm=llm,
    verbose=True,
    allow_delegation=False,
)

In [ ]:
from crewai import Task

# This prompt will be filled with user text later
raw_text = """
hi boss, i wont be able to come office tomorrow i have some personal issue.
pls approve my leave. thanks
"""

task_improve_draft = Task(
    description=(
        "You are given a rough piece of text from a user. Improve it into a clear, "
        "complete draft. Preserve meaning but fix confusion, add missing context "
        "if needed, and make it sound coherent and polite.\n\n"
        f"User text:\n{raw_text}"
    ),
    expected_output=(
        "A single, well-written draft of the text, suitable to be edited further."
    ),
    agent=draft_improver,
)

task_edit = Task(
    description=(
        "You are given an improved draft of a text (such as an email or document). "
        "Edit it to improve structure, tone, and readability. Make it professional, "
        "clear, and concise. Do not change the core meaning.\n\n"
        "Use the most recent version of the text available in the context."
    ),
    expected_output="A professionally edited version of the text.",
    agent=editor,
)

task_proofread = Task(
    description=(
        "You are given an edited version of a text. Carefully proofread it for:\n"
        "- Grammar\n- Spelling\n- Punctuation\n- Minor style issues\n\n"
        "Return the final, polished version that is ready to send or publish."
    ),
    expected_output="The final, polished text.",
    agent=proofreader,
)


In [ ]:
from crewai import Crew, Process

email_crew = Crew(
    agents=[draft_improver, editor, proofreader],
    tasks=[task_improve_draft, task_edit, task_proofread],
    process=Process.sequential,   # tasks run in order
    verbose=True,
)

result = email_crew.kickoff()

print("\n================ FINAL POLISHED TEXT ================\n")
print(result)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 4eb4d78d-2275-4fc2-bd48-0691b090ddbb                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Draft Improver                                                                                          │
│                                                                                                                 │
│  Task: You are given a rough piece of text from a user. Improve it into a clear, complete draft. Preserve       │
│  meaning but fix confusion, add missing context if needed, and make it sound coherent and polite.               │
│                                                                                                                 │
│  User text:                                                                                                     │
│                                                                                                                 │
│  hi boss, i wont be able to come office tomorrow i have some personal issue.                                    │
│  pls approve my leave. thanks                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Draft Improver                                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Dear [Boss's Name],                                                                                            │
│                                                                                                                 │
│  I am writing to inform you that I will be unable to come to the office tomorrow, [Insert Date of Absence],     │
│  due to a personal matter that requires my attention.                                                           │
│                                                                                                                 │
│  I kindly request your approval for this day of leave.                                                          │
│                                                                                                                 │
│  Thank you for your understanding.                                                                              │
│                                                                                                                 │
│  Best regards,                                                                                                  │
│                                                                                                                 │
│  [Your Name]                                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Proofreader                                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Subject: Leave Request - [Your Name] - [Date of Absence]                                                       │
│                                                                                                                 │
│  Dear [Boss's Name],                                                                                            │
│                                                                                                                 │
│  I am writing to request a day of leave for tomorrow, [Insert Date of Absence], due to a personal matter. I     │
│  will be unavailable to work on this day.                                                                       │
│                                                                                                                 │
│  I apologize for any inconvenience this may cause and appreciate your understanding and approval of this        │
│  request.                                                                                                       │
│                                                                                                                 │
│  Best regards,                                                                                                  │
│                                                                                                                 │
│  [Your Name]                                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


================ FINAL POLISHED TEXT ================

Subject: Leave Request - [Your Name] - [Date of Absence]

Dear [Boss's Name],

I am writing to request a day of leave for tomorrow, [Insert Date of Absence], due to a personal matter. I will be unavailable to work on this day.

I apologize for any inconvenience this may cause and appreciate your understanding and approval of this request.

Best regards,

[Your Name]


╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): 